In [48]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import os
import csv
from datetime import datetime
import pymysql
import json
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import cv2

In [29]:
courses = {"19AD701" : ["BDA","Priyadharshini"],
           "19AD702" : ["IoT","Shenbagarajan"],
           "19AD751" : ["BDA Lab","Priyadharshini"],
           "19AD752" : ["IoT Lab","Shenbagarajan"],
           "19ADP15" : ["SNA", "Nirmalan"],
           "19ADC15" : ["CC", "Angela"],
           "19ADC23" : ["BuDA","Nirmalan"]}

In [30]:
periods = {"Monday" : ["19AD784","19AD783","19AD783","19AD783","19ADP15","19ADC15","19AD701","19ADC14"],
           "Tuesday" : ["19AD702","19ADC23","19AD701","19AD702","asso","19AD701","19AD783","19ADC22"],
           "Wednesday" : ["19ADC15","19AD751","19AD751","19AD751","19ADP15","19ADC23","plc","plc"],
           "Thursday" : ["19AD701","19AD784","19AD702","19AD701","19ADC15","19AD702","rpp","rpp"],
           "Friday" : ["19ADP15","19AD751","19AD751","19AD751","lib","19AD784","pet","pet"],
           "Saturday" : ["19ADC15","19ADP15","19ADC22","19ADP15","19AD784","19AD702","19ADC22","club"]}

In [33]:
class EnsembleModel(nn.Module):
    def __init__(self, models):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(models)

    def forward(self, x):
        predictions = [model(x) for model in self.models]
        return torch.stack(predictions).mean(dim=0)

In [34]:
class ModifiedInceptionResnetV1(InceptionResnetV1):
    def __init__(self, *args, **kwargs):
        super(ModifiedInceptionResnetV1, self).__init__(*args, **kwargs)

        self.last_linear = nn.Linear(1792, 512) 

        self.forehead_conv1 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=2)
        self.forehead_conv2 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=2)
        
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=2)
        self.conv4 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=2)
        self.conv5 = nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=2)

    def forward(self, x):
        x = super(ModifiedInceptionResnetV1, self).forward(x)
        
        x = x.view(x.size(0), -1, 1, 1)

        x = F.relu(self.forehead_conv1(x))
        x = F.relu(self.forehead_conv2(x))
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))

        return x

In [36]:
mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20)
resnet_model_count = 3 
resnet_models = [ModifiedInceptionResnetV1(pretrained='vggface2').eval() for _ in range(resnet_model_count)]

In [46]:
ensemble_model = EnsembleModel(models=resnet_models)

In [42]:
dataset = datasets.ImageFolder('../Images/Class/')
idx_to_class = {i: c for c, i in dataset.class_to_idx.items()}

In [43]:
def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = []
name_list = []
embedding_list = []

for img, idx in loader:
        face, prob = mtcnn(img, return_prob=True)
        
        if face is not None and prob > 0.90:
            emb = ensemble_model(face.unsqueeze(0))
            embedding_list.append(emb.detach().cpu())
            name_list.append(idx_to_class[idx])


[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


KeyboardInterrupt: 

In [ ]:
data = [embedding_list, name_list]
torch.save(data, 'data.pt')

In [ ]:
# def get_current_datetime():
#     now = datetime.now()
#     date = now.strftime("%Y-%m-%d")
#     time = now.strftime("%H:%M:%S")
#     day = now.strftime("%A")
#     return date, time, day

In [ ]:
# def dbInsert(p,c,s,pn,d,t,da,pr):
#     connection = pymysql.connect(host="bxyjlxdzotlowe6fee2c-mysql.services.clever-cloud.com", port=3306, user="ugsqllp6usysqevp", passwd="E5t4uVTvTY65jaFIk5Lw", database="bxyjlxdzotlowe6fee2c")
#     cursor = connection.cursor()
#     insert1 = "INSERT INTO Attendance(PERIOD, COURSE, STAFF, STUDENT, DATE, TIME, DAY,Present) VALUES(%s, %s, %s, %s, %s, %s ,%s, %s);"
#     data = (p,c,s,pn,d,t,da,pr)

#     cursor.execute(insert1,data)
#     connection.commit()
#     connection.close()

In [ ]:
# def get_all_students_from_database():
#     connection = pymysql.connect(host="bxyjlxdzotlowe6fee2c-mysql.services.clever-cloud.com", port=3306, user="ugsqllp6usysqevp", passwd="E5t4uVTvTY65jaFIk5Lw", database="bxyjlxdzotlowe6fee2c")
#     cursor = connection.cursor()

#     cursor.execute("SELECT StudentsName FROM Students")
#     all_students = [row[0] for row in cursor.fetchall()]

#     connection.close()

#     return all_students

# all_students = get_all_students_from_database()

In [ ]:
# def course_students(course_id):
#     connection = pymysql.connect(host="bxyjlxdzotlowe6fee2c-mysql.services.clever-cloud.com", port=3306, user="ugsqllp6usysqevp", passwd="E5t4uVTvTY65jaFIk5Lw", database="bxyjlxdzotlowe6fee2c")
#     cursor = connection.cursor()

#     query = "SELECT StudentName FROM Courses WHERE SubjectId = %s"
#     cursor.execute(query, (course_id,))
#     result = cursor.fetchone()

#     cursor.close()
#     connection.close()
#     if result is not None:
#         json_array_str = result[0]
#         student_list = json.loads(json_array_str)
#         return student_list

In [ ]:
present_students = set()

In [ ]:
def predict_and_record(folder_path, data_path, output_csv):
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['Name', 'Date', 'Time', 'Day', 'Present']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        saved_data = torch.load(data_path)
        embedding_list = saved_data[0]
        name_list = saved_data[1]

        # predicted_names_set = set(name_list)
        
        for filename in os.listdir(folder_path):
            if filename.endswith(".png"):
                img_path = os.path.join(folder_path, filename)
                img = Image.open(img_path)
                face, prob = mtcnn(img, return_prob=True)
                emb = ensemble_model(face.unsqueeze(0)).detach()

                dist_list = []
                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                idx_min = dist_list.index(min(dist_list))
                predicted_name = name_list[idx_min]
                # distance = min(dist_list)

                class_dir = os.path.join("../Images/Class/", predicted_name)

                img.save(os.path.join(class_dir, filename))
                present_students.add(predicted_name)

        

In [ ]:
folder_path = '../Images/output_images/testImages1/'
data_path = 'data.pt'
output_csv = '../csvFolder/prediction.csv'

In [ ]:
predict_and_record(folder_path, data_path, output_csv)

In [ ]:
print(present_students)

{'Dharun', 'Siddarth', 'Nitheeshwaran', 'Davis', 'Salman', 'Cranio', 'Vishnu', 'Sailesh', 'Aakash'}


In [ ]:

# date, time, day = get_current_datetime()

# if day=="Monday":
#         time_split = time.split(":")
#         if(int(time_split[0])==9):
#                 period_list = periods.get("Monday")
#                 p_id = period_list[0]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==10):
#                 period_list = periods.get("Monday")
#                 p_id = period_list[1]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==11):
#                 period_list = periods.get("Monday")
#                 p_id = period_list[2]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==12):
#                 period_list = periods.get("Monday")
#                 p_id = period_list[3]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==13):
#                 period_list = periods.get("Monday")
#                 p_id = period_list[4]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==14):
#                 period_list = periods.get("Monday")
#                 p_id = period_list[5]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==15):
#                 period_list = periods.get("Monday")
#                 p_id = period_list[6]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==16):
#                 period_list = periods.get("Monday")
#                 p_id = period_list[7]
#                 student_course_list = course_students(p_id)

# if day=="Tuesday":
#         time_split = time.split(":")
#         if(int(time_split[0])==9):
#                 period_list = periods.get("Tuesday")
#                 p_id = period_list[0]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==10):
#                 period_list = periods.get("Tuesday")
#                 p_id = period_list[1]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==11):
#                 period_list = periods.get("Tuesday")
#                 p_id = period_list[2]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==12):
#                 period_list = periods.get("Tuesday")
#                 p_id = period_list[3]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==13):
#                 period_list = periods.get("Tuesday")
#                 p_id = period_list[4]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==14):
#                 period_list = periods.get("Tuesday")
#                 p_id = period_list[5]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==15):
#                 period_list = periods.get("Tuesday")
#                 p_id = period_list[6]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==16):
#                 period_list = periods.get("Tuesday")
#                 p_id = period_list[7]
#                 student_course_list = course_students(p_id)

# if day=="Wednesday":
#         time_split = time.split(":")
#         if(int(time_split[0])==9):
#                 period_list = periods.get("Wednesday")
#                 p_id = period_list[0]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==10):
#                 period_list = periods.get("Wednesday")
#                 p_id = period_list[1]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==11):
#                 period_list = periods.get("Wednesday")
#                 p_id = period_list[2]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==12):
#                 period_list = periods.get("Wednesday")
#                 p_id = period_list[3]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==13):
#                 period_list = periods.get("Wednesday")
#                 p_id = period_list[4]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==14):
#                 period_list = periods.get("Wednesday")
#                 p_id = period_list[5]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==15):
#                 period_list = periods.get("Wednesday")
#                 p_id = period_list[6]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==16):
#                 period_list = periods.get("Wednesday")
#                 p_id = period_list[7]
#                 student_course_list = course_students(p_id)

# if day=="Thursday":
#         time_split = time.split(":")
#         if(int(time_split[0])==9):
#                 period_list = periods.get("Thursday")
#                 p_id = period_list[0]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==10):
#                 period_list = periods.get("Thursday")
#                 p_id = period_list[1]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==11):
#                 period_list = periods.get("Thursday")
#                 p_id = period_list[2]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==12):
#                 period_list = periods.get("Thursday")
#                 p_id = period_list[3]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==13):
#                 period_list = periods.get("Thursday")
#                 p_id = period_list[4]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==14):
#                 period_list = periods.get("Thursday")
#                 p_id = period_list[5]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==15):
#                 period_list = periods.get("Thursday")
#                 p_id = period_list[6]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==16):
#                 period_list = periods.get("Thursday")
#                 p_id = period_list[7]
#                 student_course_list = course_students(p_id)

# if day=="Friday":
#         time_split = time.split(":")
#         if(int(time_split[0])==9):
#                 period_list = periods.get("Friday")
#                 p_id = period_list[0]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==10):
#                 period_list = periods.get("Friday")
#                 p_id = period_list[1]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==11):
#                 period_list = periods.get("Friday")
#                 p_id = period_list[2]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==12):
#                 period_list = periods.get("Friday")
#                 p_id = period_list[3]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==13):
#                 period_list = periods.get("Friday")
#                 p_id = period_list[4]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==14):
#                 period_list = periods.get("Friday")
#                 p_id = period_list[5]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==15):
#                 period_list = periods.get("Friday")
#                 p_id = period_list[6]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==16):
#                 period_list = periods.get("Friday")
#                 p_id = period_list[7]
#                 student_course_list = course_students(p_id)

# if day=="Saturday":
#         time_split = time.split(":")
#         if(int(time_split[0])==9):
#                 period_list = periods.get("Saturday")
#                 p_id = period_list[0]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==10):
#                 period_list = periods.get("Saturday")
#                 p_id = period_list[1]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==11):
#                 period_list = periods.get("Saturday")
#                 p_id = period_list[2]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==12):
#                 period_list = periods.get("Saturday")
#                 p_id = period_list[3]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==13):
#                 period_list = periods.get("Saturday")
#                 p_id = period_list[4]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==14):
#                 period_list = periods.get("Saturday")
#                 p_id = period_list[5]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==15):
#                 period_list = periods.get("Saturday")
#                 p_id = period_list[6]
#                 student_course_list = course_students(p_id)

#         elif(int(time_split[0])==16):
#                 period_list = periods.get("Saturday")
#                 p_id = period_list[7]
#                 student_course_list = course_students(p_id)



In [ ]:
# for student_name in student_course_list:
#             is_present = 1 if student_name in present_students else 0
#             date, time, day = get_current_datetime()

#             if day=="Monday":
#                     time_split = time.split(":")
#                     if(int(time_split[0])==9):
#                             period_list = periods.get("Monday")
#                             p_id = period_list[0]
#                             period = 1
#                             sub,staff = courses.get(p_id)
#                             dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                     elif(int(time_split[0])==10):
#                             period_list = periods.get("Monday")
#                             p_id = period_list[1]
#                             period = 2
#                             sub,staff = courses.get(p_id)
#                             dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                     elif(int(time_split[0])==11):
#                             period_list = periods.get("Monday")
#                             p_id = period_list[2]
#                             period = 3
#                             sub,staff = courses.get(p_id)
#                             dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                     elif(int(time_split[0])==12):
#                             period_list = periods.get("Monday")
#                             p_id = period_list[3]
#                             period = 4
#                             sub,staff = courses.get(p_id)
#                             dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                     elif(int(time_split[0])==13):
#                             period_list = periods.get("Monday")
#                             p_id = period_list[4]
#                             period = 5
#                             sub,staff = courses.get(p_id)
#                             dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                     elif(int(time_split[0])==14):
#                             period_list = periods.get("Monday")
#                             p_id = period_list[5]
#                             period = 6
#                             sub,staff = courses.get(p_id)
#                             dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                     elif(int(time_split[0])==15):
#                             period_list = periods.get("Monday")
#                             p_id = period_list[6]
#                             period = 7
#                             sub,staff = courses.get(p_id)
#                             dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                     elif(int(time_split[0])==16):
#                             period_list = periods.get("Monday")
#                             p_id = period_list[7]
#                             period = 8
#                             sub,staff = courses.get(p_id)
#                             dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#             if day=="Tuesday":
#                 time_split = time.split(":")
#                 if(int(time_split[0])==9):
#                         period_list = periods.get("Tuesday")
#                         p_id = period_list[0]
#                         period = 1
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==10):
#                         period_list = periods.get("Tuesday")
#                         p_id = period_list[1]
#                         period = 2
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==11):
#                         period_list = periods.get("Tuesday")
#                         p_id = period_list[2]
#                         period = 3
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==12):
#                         period_list = periods.get("Tuesday")
#                         p_id = period_list[3]
#                         period = 4
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==13):
#                         period_list = periods.get("Tuesday")
#                         p_id = period_list[4]
#                         period = 5
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==14):
#                         period_list = periods.get("Tuesday")
#                         p_id = period_list[5]
#                         period = 6
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==15):
#                         period_list = periods.get("Tuesday")
#                         p_id = period_list[6]
#                         period = 7
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==16):
#                         period_list = periods.get("Tuesday")
#                         p_id = period_list[7]
#                         period = 8
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#             if day=="Wednesday":
#                 time_split = time.split(":")
#                 if(int(time_split[0])==9):
#                         period_list = periods.get("Wednesday")
#                         p_id = period_list[0]
#                         period = 1
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==10):
#                         period_list = periods.get("Wednesday")
#                         p_id = period_list[1]
#                         period = 2
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==11):
#                         period_list = periods.get("Wednesday")
#                         p_id = period_list[2]
#                         period = 3
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==12):
#                         period_list = periods.get("Wednesday")
#                         p_id = period_list[3]
#                         period = 4
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==13):
#                         period_list = periods.get("Wednesday")
#                         p_id = period_list[4]
#                         period = 5
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==14):
#                         period_list = periods.get("Wednesday")
#                         p_id = period_list[5]
#                         period = 6
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==15):
#                         period_list = periods.get("Wednesday")
#                         p_id = period_list[6]
#                         period = 7
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==16):
#                         period_list = periods.get("Wednesday")
#                         p_id = period_list[7]
#                         period = 8
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#             if day=="Thursday":
#                 time_split = time.split(":")
#                 if(int(time_split[0])==9):
#                         period_list = periods.get("Thursday")
#                         p_id = period_list[0]
#                         period = 1
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==10):
#                         period_list = periods.get("Thursday")
#                         p_id = period_list[1]
#                         period = 2
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==11):
#                         period_list = periods.get("Thursday")
#                         p_id = period_list[2]
#                         period = 3
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==12):
#                         period_list = periods.get("Thursday")
#                         p_id = period_list[3]
#                         period = 4
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==13):
#                         period_list = periods.get("Thursday")
#                         p_id = period_list[4]
#                         period = 5
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==14):
#                         period_list = periods.get("Thursday")
#                         p_id = period_list[5]
#                         period = 6
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==15):
#                         period_list = periods.get("Thursday")
#                         p_id = period_list[6]
#                         period = 7
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==16):
#                         period_list = periods.get("Thursday")
#                         p_id = period_list[7]
#                         period = 8
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#             if day=="Friday":
#                 time_split = time.split(":")
#                 if(int(time_split[0])==9):
#                         period_list = periods.get("Friday")
#                         p_id = period_list[0]
#                         period = 1
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==10):
#                         period_list = periods.get("Friday")
#                         p_id = period_list[1]
#                         period = 2
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==11):
#                         period_list = periods.get("Friday")
#                         p_id = period_list[2]
#                         period = 3
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==12):
#                         period_list = periods.get("Friday")
#                         p_id = period_list[3]
#                         period = 4
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==13):
#                         period_list = periods.get("Friday")
#                         p_id = period_list[4]
#                         period = 5
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==14):
#                         period_list = periods.get("Friday")
#                         p_id = period_list[5]
#                         period = 6
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==15):
#                         period_list = periods.get("Friday")
#                         p_id = period_list[6]
#                         period = 7
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==16):
#                         period_list = periods.get("Friday")
#                         p_id = period_list[7]
#                         period = 8
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#             if day=="Saturday":
#                 time_split = time.split(":")
#                 if(int(time_split[0])==9):
#                         period_list = periods.get("Saturday")
#                         p_id = period_list[0]
#                         period = 1
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==10):
#                         period_list = periods.get("Saturday")
#                         p_id = period_list[1]
#                         period = 2
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==11):
#                         period_list = periods.get("Saturday")
#                         p_id = period_list[2]
#                         period = 3
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==12):
#                         period_list = periods.get("Saturday")
#                         p_id = period_list[3]
#                         period = 4
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==13):
#                         period_list = periods.get("Saturday")
#                         p_id = period_list[4]
#                         period = 5
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==14):
#                         period_list = periods.get("Saturday")
#                         p_id = period_list[5]
#                         period = 6
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==15):
#                         period_list = periods.get("Saturday")
#                         p_id = period_list[6]
#                         period = 7
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

#                 elif(int(time_split[0])==16):
#                         period_list = periods.get("Saturday")
#                         p_id = period_list[7]
#                         period = 8
#                         sub,staff = courses.get(p_id)
#                         dbInsert(period,p_id,staff,student_name,date,time,day,is_present)

